# 일조량에 따른 토마토 생산량과 가격 예측

- 토마토마(김준회, 연제호, 정다영, 추영은)
- 발표일 : 2020년 3월 19일

# 1. 서론

### 1) 분석 배경

최근 최악의 미세먼지 발생이 이어지면서 농작물의 피해가 속출하고 있다. 미세먼지가 햇빛을 막아 일조량이 크게 부족해져 작물의 상품성과 생산성이 뚝 떨어졌다는 뉴스가 나오고 있다. 농가에서 비닐하우스 위에 내려앉은 미세먼지를 자주 걷어내기 어렵고 유리온실도 미세먼지 등의 이유로 투광량이 떨어진다고 한다.

### 2) 분석 목적

미세먼지와 강수량에 따른 일조량의 변화와 토마토 생산량, 가격을 예측하고자 한다.

## 2. 데이터 준비

### 1) 활용 데이터

 데이터 | 사용 변수 | 시점 | 출처 | 파일 형식
:---:|:-------:|:---:|:---:|:--------:
토마토 일별 품목별 도.소매가격정보 | 품목명, 품종명,	시군구,	마켓명,	연도,	날짜,	가격 | 일 | [농산물유통정보](https://kdx.kr/kamis/periodProductList) | csv
농작물생산조사:채소생산량(과채류) | | 일 | [통계청](http://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1ET0027&vw_cd=MT_ZTITLE&list_id=F1H&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_ZTITLE) | csv
종관기상관측(ASOS) | 지점명,	일시,	평균기온,	일강수량,	합계 일조시간,	평균 전운량 | 일 | [기상청 기상자료개방포털](https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36) | csv
대기오염 최종확정자료  | 미세먼지 | 시간 | [AirKorea](https://www.airkorea.or.kr/web/last_amb_hour_data?pMENU_NO=123) | xlsx

## 3. 데이터 가공

### 1) 토마토 도매 가격 정보 전처리

In [1]:
import pandas as pd # 데이터 프레임
import numpy as np # 
import re # 정규표현식

import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
tom36 = pd.read_csv('/Users/chuu/Documents/GitHub/Learning_Spoons_Second_Project/20132016토마토_일별 품목별 도.소매가격정보.csv')
# 토마토 일별 도.소매 가격정보 2013-2016년 데이터 불러오기 
tomp36 = tom36[(tom36['시군구']!='평균') & (tom36['시군구']!='평년')]
# 데이터 하단에 있는 평균과 평년 데이터 제외
tomp36

,품목명,품종명,시군구,마켓명,연도,날짜,가격
0,토마토,토마토(10kg),서울,가락도매,2013,01/02,"29,000"
1,토마토,토마토(10kg),서울,가락도매,2013,01/03,"30,000"
2,토마토,토마토(10kg),서울,가락도매,2013,01/04,"30,000"
3,토마토,토마토(10kg),서울,가락도매,2013,01/07,"30,000"
4,토마토,토마토(10kg),서울,가락도매,2013,01/08,"27,000"
...,...,...,...,...,...,...,...
4920,토마토,토마토(10kg),대전,오정도매,2016,12/26,"40,000"
4921,토마토,토마토(10kg),대전,오정도매,2016,12/27,"40,000"
4922,토마토,토마토(10kg),대전,오정도매,2016,12/28,"40,000"
4923,토마토,토마토(10kg),대전,오정도매,2016,12/29,"40,000"


In [3]:
tom79 = pd.read_csv('/Users/chuu/Documents/GitHub/Learning_Spoons_Second_Project/20172019토마토_일별 품목별 도.소매가격정보.csv')
# 토마토 일별 도.소매 가격정보 2017-2019년 데이터 불러오기 
tomp79 = tom79[(tom79['시군구']!='평균') & (tom79['시군구']!='평년') & (tom79['연도']!= 2020)]
# 13-16년 데이터와 마찬가지로 평균 평년 데이터 제외. 2020년 데이터도 제외.
tomp79

,품목명,품종명,시군구,마켓명,연도,날짜,가격
0,토마토,토마토(10kg),서울,가락도매,2017,01/02,"43,000"
1,토마토,토마토(10kg),서울,가락도매,2017,01/03,"41,000"
2,토마토,토마토(10kg),서울,가락도매,2017,01/04,"40,000"
3,토마토,토마토(10kg),서울,가락도매,2017,01/05,"38,000"
4,토마토,토마토(10kg),서울,가락도매,2017,01/06,"38,000"
...,...,...,...,...,...,...,...
4608,토마토,토마토(10kg),대전,오정도매,2019,12/24,"34,000"
4609,토마토,토마토(10kg),대전,오정도매,2019,12/26,"31,000"
4610,토마토,토마토(10kg),대전,오정도매,2019,12/27,"30,000"
4611,토마토,토마토(10kg),대전,오정도매,2019,12/30,"27,000"


In [9]:
tomp = pd.concat([tomp36,tomp79])
# 데이터 합치기

In [10]:
tomp.shape
# 제대로 합쳐졌는지 행 개수 확인

(9323, 7)

In [11]:
tomp = tomp.drop('품목명', axis = 1)
# 품목명이 모두 토마토이고, 품종명 또한 토마토(10kg)으로 같은 정보를 담고 있으므로 품목명 컬럼을 삭제
tomp

,품종명,시군구,마켓명,연도,날짜,가격
0,토마토(10kg),서울,가락도매,2013,01/02,"29,000"
1,토마토(10kg),서울,가락도매,2013,01/03,"30,000"
2,토마토(10kg),서울,가락도매,2013,01/04,"30,000"
3,토마토(10kg),서울,가락도매,2013,01/07,"30,000"
4,토마토(10kg),서울,가락도매,2013,01/08,"27,000"
...,...,...,...,...,...,...
4608,토마토(10kg),대전,오정도매,2019,12/24,"34,000"
4609,토마토(10kg),대전,오정도매,2019,12/26,"31,000"
4610,토마토(10kg),대전,오정도매,2019,12/27,"30,000"
4611,토마토(10kg),대전,오정도매,2019,12/30,"27,000"


In [12]:
tomp = tomp.sort_values(by = ['시군구','마켓명', '연도', '날짜'])
tomp
#시군구, 마켓명, 연도, 날짜 오름차순으로 정렬

,품종명,시군구,마켓명,연도,날짜,가격
2955,토마토(10kg),광주,각화도매,2013,01/02,"28,000"
2956,토마토(10kg),광주,각화도매,2013,01/03,"28,000"
2957,토마토(10kg),광주,각화도매,2013,01/04,"28,000"
2958,토마토(10kg),광주,각화도매,2013,01/07,"28,000"
2959,토마토(10kg),광주,각화도매,2013,01/08,"28,000"
...,...,...,...,...,...,...
728,토마토(10kg),서울,가락도매,2019,12/24,"30,000"
729,토마토(10kg),서울,가락도매,2019,12/26,"30,000"
730,토마토(10kg),서울,가락도매,2019,12/27,"30,000"
731,토마토(10kg),서울,가락도매,2019,12/30,"24,000"


In [13]:
tomp.to_csv('토마토 도소매 가격 데이터 2013-2019.csv', index = False, encoding = 'CP949')
# 2013-2019 토마토 도소매 가격 데이터 csv파일로 저장

In [14]:
pd.read_csv('토마토 도소매 가격 데이터 2013-2019.csv', encoding = 'CP949')
# 파일 불러와 확인하기

,품종명,시군구,마켓명,연도,날짜,가격
0,토마토(10kg),광주,각화도매,2013,01/02,"28,000"
1,토마토(10kg),광주,각화도매,2013,01/03,"28,000"
2,토마토(10kg),광주,각화도매,2013,01/04,"28,000"
3,토마토(10kg),광주,각화도매,2013,01/07,"28,000"
4,토마토(10kg),광주,각화도매,2013,01/08,"28,000"
...,...,...,...,...,...,...
9318,토마토(10kg),서울,가락도매,2019,12/24,"30,000"
9319,토마토(10kg),서울,가락도매,2019,12/26,"30,000"
9320,토마토(10kg),서울,가락도매,2019,12/27,"30,000"
9321,토마토(10kg),서울,가락도매,2019,12/30,"24,000"


### 2) 농작물생산조사:채소생산량(과채류)	

In [2]:
# 토마토 생산량 파일 불러오기
to = pd.read_csv('/Users/jungdayoung/Documents/GitHub/Learning_Spoons_Second_Project/채소생산량_과채류__20200304201937.csv', encoding='euc-kr', header=0)



In [3]:
# 토마토 생산량 파일 확인
to.head(1)

,시도별,2013,2013.1,2013.2,2013.3,2013.4,2013.5,2013.6,2013.7,2014,...,2017.6,2017.7,2018,2018.1,2018.2,2018.3,2018.4,2018.5,2018.6,2018.7
0,시도별,토마토:면적 (ha),생산량 (톤),노지토마토:면적 (ha),10a당 생산량 (kg),생산량 (톤),시설토마토:면적 (ha),10a당 생산량 (kg),생산량 (톤),토마토:면적 (ha),...,10a당 생산량 (kg),생산량 (톤),토마토:면적 (ha),생산량 (톤),노지토마토:면적 (ha),10a당 생산량 (kg),생산량 (톤),시설토마토:면적 (ha),10a당 생산량 (kg),생산량 (톤)


In [4]:
# 컬럼명의 .1, .2 ...을 수정하기 위해 컬럼명을 리스트에 담기
to_cols = to.columns.to_list()

# for문을 통해 연도의 4글자만 가져오기
for i in range(len(to_cols)):
    if len(to_cols[i]) > 4:
        to_cols[i] = to_cols[i][:4]

In [5]:
# 수정한 연도를 컬럼으로 지정하기
to.columns = to_cols

# 수정한 컬럼명 확인하기
to.head(2)

,시도별,2013,2013,2013,2013,2013,2013,2013,2013,2014,...,2017,2017,2018,2018,2018,2018,2018,2018,2018,2018
0,시도별,토마토:면적 (ha),생산량 (톤),노지토마토:면적 (ha),10a당 생산량 (kg),생산량 (톤),시설토마토:면적 (ha),10a당 생산량 (kg),생산량 (톤),토마토:면적 (ha),...,10a당 생산량 (kg),생산량 (톤),토마토:면적 (ha),생산량 (톤),노지토마토:면적 (ha),10a당 생산량 (kg),생산량 (톤),시설토마토:면적 (ha),10a당 생산량 (kg),생산량 (톤)
1,계,6054,388624,0,0,0,6054,6419,388624,7070,...,6142,355107,6058,388657,0,0,0,6058,6415,388657


In [6]:
# 컬럼을 리스트에 담기
to_cols = to.columns.to_list()

# 첫 번째 행을 to_row에 담기
to_row = to.iloc[0]

# melt를 사용하기 위해 컬럼명 사이에 공백 넣기
to_cols = [f'{i} ' for i in to_cols]
tom = to_cols + to_row

# 리스트 확인하기
tom.head()

시도별                시도별 시도별
2013      2013 토마토:면적 (ha)
2013          2013 생산량 (톤)
2013    2013 노지토마토:면적 (ha)
2013    2013 10a당 생산량 (kg)
Name: 0, dtype: object

In [7]:
# 첫 번째 인덱스명을 '연도'로 수정하기
tom[0] = "연도"

# 첫 번째 컬럼명을 '연도'로 수정하기
to.columns = tom

# 수정한 컬럼명 확인하기
to.head(2)

,연도,2013 토마토:면적 (ha),2013 생산량 (톤),2013 노지토마토:면적 (ha),2013 10a당 생산량 (kg),2013 생산량 (톤),2013 시설토마토:면적 (ha),2013 10a당 생산량 (kg),2013 생산량 (톤),2014 토마토:면적 (ha),...,2017 10a당 생산량 (kg),2017 생산량 (톤),2018 토마토:면적 (ha),2018 생산량 (톤),2018 노지토마토:면적 (ha),2018 10a당 생산량 (kg),2018 생산량 (톤),2018 시설토마토:면적 (ha),2018 10a당 생산량 (kg),2018 생산량 (톤)
0,시도별,토마토:면적 (ha),생산량 (톤),노지토마토:면적 (ha),10a당 생산량 (kg),생산량 (톤),시설토마토:면적 (ha),10a당 생산량 (kg),생산량 (톤),토마토:면적 (ha),...,10a당 생산량 (kg),생산량 (톤),토마토:면적 (ha),생산량 (톤),노지토마토:면적 (ha),10a당 생산량 (kg),생산량 (톤),시설토마토:면적 (ha),10a당 생산량 (kg),생산량 (톤)
1,계,6054,388624,0,0,0,6054,6419,388624,7070,...,6142,355107,6058,388657,0,0,0,6058,6415,388657


In [8]:
# 사용하지 않는 0, 1 행 삭제하기
to = to.drop(labels=[0,1],axis=0)

# 인덱스 리셋하기
to.reset_index(drop=True)

# 수정한 인덱스 확인
to.head()

,연도,2013 토마토:면적 (ha),2013 생산량 (톤),2013 노지토마토:면적 (ha),2013 10a당 생산량 (kg),2013 생산량 (톤),2013 시설토마토:면적 (ha),2013 10a당 생산량 (kg),2013 생산량 (톤),2014 토마토:면적 (ha),...,2017 10a당 생산량 (kg),2017 생산량 (톤),2018 토마토:면적 (ha),2018 생산량 (톤),2018 노지토마토:면적 (ha),2018 10a당 생산량 (kg),2018 생산량 (톤),2018 시설토마토:면적 (ha),2018 10a당 생산량 (kg),2018 생산량 (톤)
2,서울특별시,2,137,0,0,0,2,6850,137,1,...,1433,34,4,18,0,0,0,4,450,18
3,부산광역시,355,22056,0,0,0,355,6213,22056,423,...,6081,19196,363,25248,0,0,0,363,6952,25248
4,대구광역시,98,7933,0,0,0,98,8095,7933,71,...,5174,10201,167,11372,0,0,0,167,6800,11372
5,인천광역시,111,5130,0,0,0,111,4622,5130,120,...,7023,3887,54,3491,0,0,0,54,6515,3491
6,광주광역시,260,5613,0,0,0,260,2159,5613,216,...,7679,9751,123,11753,0,0,0,123,9538,11753


In [9]:
# 롱 포맷으로 변경하기 위해 melt 함수 사용하기
tos = pd.melt(to, id_vars='연도')

# 롱 포맷으로 변경된 데이터 확인
tos.head()

,연도,0,value
0,서울특별시,2013 토마토:면적 (ha),2
1,부산광역시,2013 토마토:면적 (ha),355
2,대구광역시,2013 토마토:면적 (ha),98
3,인천광역시,2013 토마토:면적 (ha),111
4,광주광역시,2013 토마토:면적 (ha),260


In [10]:
# 0과 '연도'를 기준으로 정렬하기
toss = tos.groupby([0,'연도'])['value'].sum()

# 정렬한 데이터를 데이터프레임으로 만들기
toss = pd.DataFrame(toss)


# 데이터프레임 확인하기
toss.head()

value
0                  연도          
2013 10a당 생산량 (kg) 강원도    06245
                   경기도    06606
                   경상남도   07231
                   경상북도   06707
                   광주광역시  02159

In [11]:
# 인덱스 리셋하기
toss = toss.reset_index()

# 컬럼명을 '항목', '시군구'로 변경하기
toss.rename(columns={0:'항목','연도':'시군구'},inplace=True)

# 변경된 데이터프레임 확인
toss.head()

,항목,시군구,value
0,2013 10a당 생산량 (kg),강원도,06245
1,2013 10a당 생산량 (kg),경기도,06606
2,2013 10a당 생산량 (kg),경상남도,07231
3,2013 10a당 생산량 (kg),경상북도,06707
4,2013 10a당 생산량 (kg),광주광역시,02159


In [12]:
# '연도' 컬럼 만들기
toss['연도'] = toss['항목'].str[:4]

# '항목' 컬럼에 '연도'를 제외한 내용 남기기
toss['항목']=toss['항목'].str[4:]

# 수정된 내용 확인
toss.head()

,항목,시군구,value,연도
0,10a당 생산량 (kg),강원도,06245,2013
1,10a당 생산량 (kg),경기도,06606,2013
2,10a당 생산량 (kg),경상남도,07231,2013
3,10a당 생산량 (kg),경상북도,06707,2013
4,10a당 생산량 (kg),광주광역시,02159,2013


In [13]:
toss.head(16)

,항목,시군구,value,연도
0,10a당 생산량 (kg),강원도,06245,2013
1,10a당 생산량 (kg),경기도,06606,2013
2,10a당 생산량 (kg),경상남도,07231,2013
3,10a당 생산량 (kg),경상북도,06707,2013
4,10a당 생산량 (kg),광주광역시,02159,2013
5,10a당 생산량 (kg),대구광역시,08095,2013
6,10a당 생산량 (kg),대전광역시,05016,2013
7,10a당 생산량 (kg),부산광역시,06213,2013
8,10a당 생산량 (kg),서울특별시,06850,2013
9,10a당 생산량 (kg),세종특별자치시,--,2013


In [14]:
toss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 4 columns):
항목       510 non-null object
시군구      510 non-null object
value    510 non-null object
연도       510 non-null object
dtypes: object(4)
memory usage: 16.1+ KB


In [22]:
# value를 숫자로 변경 시 오류 발생 -> 세종특별자치시에 '-'값이 있기 때문
# '-'를 0으로 변경
toss.loc[toss['시군구'].str.contains('세종'),'value'] = toss.loc[toss['시군구'].str.contains('세종'),'value'].str.replace(r'-+','0')

# 'value' 컬럼을 숫자 타입으로 변경
toss['value'] = toss['value'].astype(int)

# 변경된 데이터 확인
toss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 4 columns):
항목       510 non-null object
시군구      510 non-null object
value    510 non-null int64
연도       510 non-null object
dtypes: int64(1), object(3)
memory usage: 16.1+ KB


In [36]:
toss.head(20)

,항목,시군구,value,연도
0,10a당 생산량 (kg),강원도,6245,2013
1,10a당 생산량 (kg),경기도,6606,2013
2,10a당 생산량 (kg),경상남도,7231,2013
3,10a당 생산량 (kg),경상북도,6707,2013
4,10a당 생산량 (kg),광주광역시,2159,2013
5,10a당 생산량 (kg),대구광역시,8095,2013
6,10a당 생산량 (kg),대전광역시,5016,2013
7,10a당 생산량 (kg),부산광역시,6213,2013
8,10a당 생산량 (kg),서울특별시,6850,2013
9,10a당 생산량 (kg),세종특별자치시,0,2013


In [78]:
pd.pivot_table(toss, index= '연도', columns='항목', aggfunc='sum')

value                                                          
항목    10a당 생산량 (kg)  노지토마토:면적 (ha)        생산량 (톤)  시설토마토:면적 (ha)  토마토:면적 (ha)
연도                                                                           
2013          97299              0   358919258624           6054         6054
2014          99412              0  1472247968960           7070         7070
2015          98864              0   442608826982           6976         6976
2016          99472              0   380156970303           6391         6391
2017          97133              0   335358529107           5781         5781
2018         101517              0   380223236657           6059         6059

In [133]:
pto = pd.pivot_table(toss, index= '연도', columns='항목', aggfunc='sum')
pto.columns = pto.columns.levels[1]

In [134]:
pto = pto.reset_index(level=0)

In [135]:
new_col = pto['연도'].values

In [136]:
pto

항목,연도,10a당 생산량 (kg),노지토마토:면적 (ha),생산량 (톤),시설토마토:면적 (ha),토마토:면적 (ha)
0,2013,97299,0,358919258624,6054,6054
1,2014,99412,0,1472247968960,7070,7070
2,2015,98864,0,442608826982,6976,6976
3,2016,99472,0,380156970303,6391,6391
4,2017,97133,0,335358529107,5781,5781
5,2018,101517,0,380223236657,6059,6059


In [140]:
pto.reset_index(drop=True,col_level=0)

항목,연도,10a당 생산량 (kg),노지토마토:면적 (ha),생산량 (톤),시설토마토:면적 (ha),토마토:면적 (ha)
0,2013,97299,0,358919258624,6054,6054
1,2014,99412,0,1472247968960,7070,7070
2,2015,98864,0,442608826982,6976,6976
3,2016,99472,0,380156970303,6391,6391
4,2017,97133,0,335358529107,5781,5781
5,2018,101517,0,380223236657,6059,6059


### 3) 종관기상관측 전처리

In [ ]:
# 
df = pd.read_csv('/Users/jungdayoung/Documents/GitHub/Learning_Spoons_Second_Project/OBS_ASOS_DD_20200304204217.csv', encoding='euc-kr')


In [ ]:
df.head(2)

In [ ]:
df.columns

In [ ]:
df_s = df[['지점명', '일시', '평균기온(°C)', '일강수량(mm)', '합계 일조시간(hr)', '평균 전운량(1/10)']]
df_s.head()

In [ ]:
df_s.info()

## 4. 머신러닝 모델 적용

### 1) 시계열?

### 2) 랜덤포레스트